# Hierarchical Indexing

Hierarchical indexing pada dasarnya adalah banyak index yang membentuk hirarki.

In [1]:
import pandas as pd
import numpy as np

## A Multiply Indexed Series

Perhatikan contoh-contoh berikut:

### The bad way

Cara yang salah dalam membuat hirarki index:

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

Berdasarkan hasil di atas proses selection dan slicing masih dapat dilakukan dengan relatif mudah. Namun bagaimana jika kita ingin mengambil semua nilai pada tahun 2010? Perhatikan bagaimana sulitnya melakukan hal tersebut.

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### The Better Way: Pandas MultiIndex

Hal di atas dapat dengan mudah dilakukan dengan mengimplementasikan kelas **`MultiIndex`** pada Pandas, karena data awal berasal dari struktur tuple maka kita dapat memanggil method **`from_tuples()`** kemudian memasukkan index dari data sebagai parameter.

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex([('California', 2000),
            ('California', 2010),
            (  'New York', 2000),
            (  'New York', 2010),
            (     'Texas', 2000),
            (     'Texas', 2010)],
           )

Selanjutnya kita mengimplementasikan method **`re-index`** untuk menghasilkan ``MultiIndex``:

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

Perhatikan perbedaan kode program dengan kode program sebelumnya jika ingin mengambil data pada tahun 2010 saja.

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndex as extra dimension

Method ``unstack()`` dapat mengubah multiply indexed ``Series`` menjadi index pada ``DataFrame`` biasa:

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


Method ``stack()`` melakukan hal yang sebaliknya:

In [10]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

In [11]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18.unstack()

,2000,2010
California,0.273594,0.249211
New York,0.247010,0.222831
Texas,0.283251,0.273568


## Methods of MultiIndex Creation

In [11]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.730376  0.984887
  2  0.376737  0.369803
b 1  0.453445  0.926283
  2  0.358879  0.253801

In [12]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### Explicit MultiIndex constructors


In [13]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [14]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

In [15]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           )

### MultiIndex level names

In [17]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### MultiIndex for columns

In [18]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  37.6  27.0  37.0  47.0  37.2
     2      21.0  35.4  26.0  37.6  48.0  36.1
2014 1      31.0  37.7  33.0  37.1  37.0  36.8
     2      32.0  36.1  33.0  37.9  34.0  36.1

In [19]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      27.0  37.0
     2      26.0  37.6
2014 1      33.0  37.1
     2      33.0  37.9

## Indexing and Slicing a MultiIndex

### Multiply indexed Series

In [20]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [21]:
pop['California', 2000]

33871648

In [22]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [23]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [24]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [25]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [26]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Multiply indexed DataFrames

In [27]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      32.0  37.6  27.0  37.0  47.0  37.2
     2      21.0  35.4  26.0  37.6  48.0  36.1
2014 1      31.0  37.7  33.0  37.1  37.0  36.8
     2      32.0  36.1  33.0  37.9  34.0  36.1

In [28]:
health_data['Guido', 'HR']

year  visit
2013  1        27.0
      2        26.0
2014  1        33.0
      2        33.0
Name: (Guido, HR), dtype: float64

In [29]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      32.0  37.6
     2      21.0  35.4

In [30]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        32.0
      2        21.0
2014  1        31.0
      2        32.0
Name: (Bob, HR), dtype: float64

In [33]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,31.0,32.0,35.0
2014,1,30.0,39.0,61.0


### Index setting and resetting

In [32]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


In [34]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561